# A Monte Carlo program for the Newman-Moore triangular lattice Ising model

In [1]:
using DelimitedFiles
using Random; Random.seed!(0);
const seed = 1111;
rng = MersenneTwister(seed);

const EqSteps = 100
const NumSteps = 1000 #Monte Carlo production steps

#k = 2
#L = 2^k
L = 65
N = L*L
Spin = fill(1,N);

In [2]:
#This is the data structure that relates up-triangles to spin indices
function make_up_triangle(L, N)
    UpTriangle = zeros(Int,N,3)
    for i = 1:N
        UpTriangle[i,1] = i
        UpTriangle[i,2] = i+1    
        UpTriangle[i,3] = i+L
        #fix PBCs
        if mod(i,L) == 0
            UpTriangle[i,2] = i + 1 - L
        end
        if (i+L)> N
            UpTriangle[i,3] = i + L - N
        end
    end
    return UpTriangle
end
UpTriangle = make_up_triangle(L, N);

In [3]:
#This is the inverse data structure that relates a spin index to its 3 up-triangles
function make_associated_tri(L, N)
    AssociatedTri = zeros(Int,N,3)
    for i = 1:N
        AssociatedTri[i,1] = i
        AssociatedTri[i,2] = i-1
        AssociatedTri[i,3] = i-L
        if mod(i-1+L,L) == 0
            AssociatedTri[i,2] = i - 1 + L
        end
        if i<(L+1) 
            AssociatedTri[i,3] = i + N - L
        end
    end
    return AssociatedTri
end
AssociatedTri = make_associated_tri(L, N);

In [4]:
#here is the brute force calculation of the energy
function Energy_Total(Energy,Spin)
    Energy = 0
    for i = 1:N
        Spin1 = Spin[UpTriangle[i,1]]
        Spin2 = Spin[UpTriangle[i,2]]
        Spin3 = Spin[UpTriangle[i,3]]
        Energy += Spin1 * Spin2 * Spin3  #J = 1
        end # limit all arithmetic to integers for as long as we can
    return Energy / 2
end #Energy_Total

Energy_Total (generic function with 1 method)

In [5]:
#here is the energy DIFFERENCE calculated from the local triangular plaquettes
function Energy_Diff(Spin,spin_index)
    
    Tri1 = AssociatedTri[spin_index,1]
    Tri2 = AssociatedTri[spin_index,2]
    Tri3 = AssociatedTri[spin_index,3]

    plaquette_factor = 0
    plaquette_factor -= Spin[UpTriangle[Tri1,2]] * Spin[UpTriangle[Tri1,3]]
    plaquette_factor -= Spin[UpTriangle[Tri2,1]] * Spin[UpTriangle[Tri2,3]]
    plaquette_factor -= Spin[UpTriangle[Tri3,1]] * Spin[UpTriangle[Tri3,2]]
    
    return Spin[spin_index] * plaquette_factor
end

# Energy_Diff can only take 4 values: -3, -1, 1, 3
# BUT, the non-positive values always result in acceptance
# so we only need to cache the value of exp(-dE / T) for 2 values of dE
# we'll store these values in a tuple of length 3, and get the
# acceptance prob by indexing with dE

Energy_Diff (generic function with 1 method)

In [6]:
function MetropolisAccept(DeltaE::Int, acceptance_ratios::NTuple{3, Float64})::Bool
    if DeltaE <= 0
        return true
    else
        rnum = rand(rng)  #random number for Metropolis
        if (acceptance_ratios[DeltaE] > rnum)
            return true
        end
    end
    return false
end

MetropolisAccept (generic function with 1 method)

In [7]:
Tarr = zeros(Float64,0)  
EMC = zeros(Float64,0)  
SpecHeat = zeros(Float64,0)  
Eexact = zeros(Float64,0)
Acceptance = zeros(Float64,0) 

const autocorr_steps = 100
const data_samples = 5000
const num_autocorrs = data_samples - autocorr_steps + 1

const gamma = 0.01
const stoptime = 200
for time = 0:stoptime
#for T = 10:-0.1:0.1  #count down  
    
    print(time," ")
    T = exp(-gamma*time)
    beta = 1.0/T
    
    acceptance_ratios = (exp(-beta*1), exp(-beta*2), exp(-beta*3))

    #initialize the energy
    Energy = 0
    Energy=Energy_Total(Energy,Spin)
    #println(Energy)

    for step = 1:EqSteps
        for i = 1:N  #multiple single spin flips
            spin_i = rand(1:N)
            DeltaE = Energy_Diff(Spin,spin_i)
            if MetropolisAccept(DeltaE, acceptance_ratios)
                Energy += DeltaE
                Spin[spin_i] *= -1
            end 
        end #i
    end #Equilibration
    
    E_avg = 0
    E2 = 0
    A_rate = 0
    for step = 1:NumSteps
        Accept = 0
        for i = 1:N  #multiple single spin flips
            spin_i = rand(1:N)
            DeltaE = Energy_Diff(Spin,spin_i)
            if MetropolisAccept(DeltaE, acceptance_ratios)
                Energy += DeltaE
                Spin[spin_i] *= -1
                Accept += 1
                end  
        end #i
        E_avg += Energy
        E2 += Energy*Energy
        A_rate += Accept/N
    end #Monte Carlo production step

    E_exact = 1/(1+exp(beta))
    Cv = E2/NumSteps - (E_avg/NumSteps)^2
    println(T," ",E_avg/NumSteps/N," ",E_exact-0.5," ",Cv/N/T/T," ",A_rate/NumSteps)
     
    push!(Tarr,T)
    push!(EMC,E_avg/NumSteps/N)
    push!(SpecHeat,Cv/(T*T*N))
    push!(Eexact,E_exact-0.5)
    push!(Acceptance,A_rate/NumSteps)
    
    #Let's write configuration files for data
    if time == stoptime
        println("Starting to get final data")        
        Tstring = first(string(T),8)
        filename = string("stewart_data/L", L, "/gamma", gamma, "_stoptime", stoptime,
            "_T", Tstring, "_seed", seed, ".dat")
        mag_filename = string("stewart_data/L", L, "/gamma", gamma, "_stoptime", stoptime,
            "_T", Tstring, "_seed", seed, "_mag.dat")
        
        autocorrs = zeros(autocorr_steps)
        Mag_array = zeros(data_samples)
        for step = 1:data_samples #configuration data files
            for i = 1:20*N  #multiple single spin flips
                spin_i = rand(1:N)
                DeltaE = Energy_Diff(Spin,spin_i)
                if MetropolisAccept(DeltaE, acceptance_ratios)
                    Energy += DeltaE
                    Spin[spin_i] *= -1
                    end 
                end #i
            Mag_array[step] = sum(Spin)

            open(filename, "a") do io
                writedlm(io, Spin' , " ")
                end # end open
            end #step 

        
        println("Writing magnetization data")
        
        # Save data with magnetizations
        open(mag_filename, "w") do io
            writedlm(io, Mag_array', " ")
            end #end open
        
    end #if stoptime

end #T

EMC_filename = string("stewart_data/L", L, "/gamma", gamma, "_stoptime", stoptime,
        "_seed", seed, "_energy.dat")
open(EMC_filename, "w") do io
    writedlm(io, EMC', " ")
end #end open

temp_filename = string("stewart_data/L", L, "/gamma", gamma, "_stoptime", stoptime, "_temperature.dat")
open(temp_filename, "w") do io
    writedlm(io, Tarr', " ")
end #end open

0 1.0 -0.23110059171597633 -0.2310585786300049 0.19246816568049058 0.3561730177514794
1 0.9900498337491681 -0.23347810650887574 -0.2330299669032307 0.20670558324465232 0.3509746745562128
2 0.9801986733067553 -0.23490958579881657 -0.2350118162075338 0.20576544501080363 0.34701846153846194
3 0.9704455335485082 -0.2365491124260355 -0.23700400723120663 0.20770367293222172 0.3424068639053255
4 0.9607894391523232 -0.23897207100591716 -0.23900641521388233 0.20182326528560043 0.33776923076923077
5 0.951229424500714 -0.2411955029585799 -0.2410189098634476 0.20425583160682412 0.33293254437869796
6 0.9417645335842487 -0.2433978698224852 -0.24304135527459575 0.21907204155583382 0.3280684023668645
7 0.9323938199059483 -0.24509585798816566 -0.24507360984918386 0.21789455522741025 0.32388426035502926
8 0.9231163463866358 -0.24661964497041422 -0.2471155262185617 0.22894071522523116 0.3195410650887574
9 0.9139311852712282 -0.24917230769230767 -0.24916695116804632 0.21555324553748229 0.31493514792899413

81 0.4448580662229411 -0.4051867455621302 -0.4044699295088948 0.3992773812329979 0.05071313609467455
82 0.44043165450599925 -0.40766011834319527 -0.4064042004068251 0.4883702605079057 0.048374437869822436
83 0.43604928632153556 -0.40808402366863905 -0.408322186256602 0.4301979030753727 0.04746911242603556
84 0.4317105234290797 -0.4100596449704142 -0.4102235042035662 0.35359076839450915 0.045414674556213036
85 0.4274149319487267 -0.41214390532544376 -0.4121077784057653 0.3254376852186316 0.04350816568047341
86 0.4231620823177488 -0.4133389349112426 -0.4139746404176587 0.43699503731363026 0.04200520710059174
87 0.418951549247639 -0.4156627218934911 -0.41582372956939223 0.49584468632887313 0.040185562130177474
88 0.4147829116815814 -0.4159779881656805 -0.4176546933409021 0.3414644647436373 0.039036449704142016
89 0.41065575275234545 -0.42094035502958577 -0.4194671877301032 0.38049926662681965 0.03625846153846149
90 0.4065696597405991 -0.41918721893491123 -0.42126087761441716 0.38445924019

160 0.20189651799465538 -0.4808452071005917 -0.492987560019391 0.015085090561436986 0.0007805917159763231
161 0.1998876140751445 -0.4810610650887574 -0.4933258123831239 0.012254662139842819 0.0007226035502958516
162 0.19789869908361465 -0.4810397633136095 -0.4936510100837256 0.019844089364933815 0.0006989349112425971
163 0.1959295741269093 -0.48082934911242603 -0.49396349163990927 0.01679491654218663 0.0007088757396449644
164 0.19398004229089189 -0.4812968047337278 -0.49426359421358085 0.011187356144636092 0.0006085207100591666
165 0.1920499086207541 -0.4812953846153846 -0.4945516532790027 0.010397591142905836 0.0005827218934911203
166 0.1901389801015205 -0.4812144378698225 -0.494828002302305 0.010899415470488057 0.0005772781065088723
167 0.18824706563874677 -0.48131100591715975 -0.4950929724317374 0.010188801228208892 0.0005450887573964473
168 0.18637397603940997 -0.4815081656804734 -0.49534689219902056 0.008528523532261181 0.0005235502958579852
169 0.18451952399298926 -0.481008284023

In [8]:
using Plots
Lstring = string("L=",L)
plot(Tarr, [EMC, Eexact], title = Lstring, label = ["Monte Carlo" "Exact"], lw = 3 )
xlabel!("Temperature")
ylabel!("Energy")

ArgumentError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.


In [9]:
autocorr = (M_t_M_ti_avg .- M_t_avg * M_ti_avg) ./ (M_t_stdev * M_ti_stdev)
plot(1:100, autocorr, title="Magnetization Autocorrelation", label="L = 64", lw=3)
xlabel!("MC Step")
ylabel!("Autocorrelation")

UndefVarError: UndefVarError: M_t_avg not defined

In [10]:
plot(Tarr, Acceptance, label = Lstring, lw = 3 )
xlabel!("Temperature")
ylabel!("Acceptance Rate")

UndefVarError: UndefVarError: Lstring not defined